## Generating correlation matrice for supplementary figure 1

package loading

In [1]:
library(Seurat)
library(scran)
library(bluster)
library(dplyr)
library(tidyr)
library(tibble)
library(ggplot2)

Warning message:
“le package ‘Seurat’ a été compilé avec la version R 4.1.2”
Attaching SeuratObject

Le chargement a nécessité le package : SingleCellExperiment

Le chargement a nécessité le package : SummarizedExperiment

Le chargement a nécessité le package : MatrixGenerics

Le chargement a nécessité le package : matrixStats


Attachement du package : ‘MatrixGenerics’


Les objets suivants sont masqués depuis ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs

Loading the Seurat object

In [2]:
embryo <- readRDS("/home/adufour/work/rds_storage/embryo_all_states.rds")

Calculate PCA for all cells

In [3]:
embryo <- NormalizeData(embryo) %>% FindVariableFeatures() %>% ScaleData() %>% RunPCA(verbose = FALSE)

Centering and scaling data matrix



In [4]:
embryo@meta.data$time_cell_type <- paste0(embryo@meta.data$TIME, "_", embryo@meta.data$cell_type)

Rename labels for plotting

In [5]:
embryo@meta.data$time_cell_type[embryo@meta.data$time_cell_type == "J5_ICM"] <- "D5 Inner Cell Mass"
embryo@meta.data$time_cell_type[embryo@meta.data$time_cell_type == "J5_Trophectoderme"] <- "D5 Trophectoderm"
embryo@meta.data$time_cell_type[embryo@meta.data$time_cell_type == "J7_Epiblaste"] <- "D7 Epiblast"
embryo@meta.data$time_cell_type[embryo@meta.data$time_cell_type == "J9_Epiblaste"] <- "D9 Epiblast"
embryo@meta.data$time_cell_type[embryo@meta.data$time_cell_type == "J11_Epiblaste"] <- "D11 Epiblast"
embryo@meta.data$time_cell_type[embryo@meta.data$time_cell_type == "J7_Trophectoderme"] <- "D7 Trophectoderm"
embryo@meta.data$time_cell_type[embryo@meta.data$time_cell_type == "J9_Trophectoderme"] <- "D9 Trophectoderm A"
embryo@meta.data$time_cell_type[embryo@meta.data$time_cell_type == "J9_Trophectoderme ?"] <- "D9 Trophectoderm B"
embryo@meta.data$time_cell_type[embryo@meta.data$time_cell_type == "J11_Trophectoderme 1"] <- "D11 Trophectoderm A"
embryo@meta.data$time_cell_type[embryo@meta.data$time_cell_type == "J11_Trophectoderme 2"] <- "D11 Trophectoderm B"
embryo@meta.data$time_cell_type[embryo@meta.data$time_cell_type == "J11_Trophectoderme 3"] <- "D11 Trophectoderm C"
embryo@meta.data$time_cell_type[embryo@meta.data$time_cell_type == "J7_Endoderme"] <- "D7 Hypoblast"
embryo@meta.data$time_cell_type[embryo@meta.data$time_cell_type == "J9_Endoderme"] <- "D9 Hypoblast"
embryo@meta.data$time_cell_type[embryo@meta.data$time_cell_type == "J11_Endoderme 1"] <- "D11 Hypoblast A"
embryo@meta.data$time_cell_type[embryo@meta.data$time_cell_type == "J11_Endoderme 2"] <- "D11 Hypoblast B"
embryo@meta.data$time_cell_type[embryo@meta.data$time_cell_type == "J11_Endoderme 3"] <- "D11 Hypoblast C"
embryo@meta.data$time_cell_type[embryo@meta.data$time_cell_type == "J11_Mesendoderme"] <- "D11 Mesendoderm"

Set order for plotting

In [6]:
embryo$order <- factor(embryo$time_cell_type, levels = c("D5 Inner Cell Mass", "D5 Trophectoderm", "D7 Epiblast", "D7 Trophectoderm", "D7 Hypoblast",
                                                                              "D9 Epiblast", "D9 Trophectoderm A", "D9 Trophectoderm B", "D9 Hypoblast", 
                                                                              "D11 Epiblast", "D11 Trophectoderm A", "D11 Trophectoderm B",  "D11 Trophectoderm C",
                                                                               "D11 Hypoblast A", "D11 Hypoblast B", "D11 Hypoblast C", "D11 Mesendoderm"),ordered = TRUE)

convert to singlecellexperiment format

In [7]:
sce <- as.SingleCellExperiment(embryo)
reducedDim(sce, 'PCA_sub') <- reducedDim(sce, 'PCA')[,1:25, drop = FALSE]

Calculate scran similarity

In [8]:
g <- scran::buildSNNGraph(sce, use.dimred = 'PCA_sub')
ratio <- bluster::pairwiseModularity(g, embryo@meta.data$order, as.ratio = TRUE)
ratio_to_plot <- log10(ratio+1)

Plotting

In [9]:
plots <- ratio_to_plot %>%
  as_tibble(rownames = NA) %>%
  rownames_to_column(var = 'cluster_1') %>%
  pivot_longer(
    cols = 2:ncol(.),
    names_to = 'cluster_2',
    values_to = 'probability'
  ) %>%
  mutate(
    cluster_1 = as.character(cluster_1),
    cluster_1 = factor(cluster_1, levels = rev(unique(cluster_1))),
    cluster_2 = factor(cluster_2, levels = unique(cluster_2))
  ) %>%
  ggplot(aes(cluster_2, cluster_1, fill = probability)) +
  geom_tile(color = 'white') +
  geom_text(aes(label = round(probability, digits = 2)), size = 7.5) +
  scale_x_discrete(name = 'Cluster', position = 'top') +
  scale_y_discrete(name = 'Cluster') +
  scale_fill_gradient(
    name = 'log10(ratio)', low = 'white', high = '#c0392b', na.value = '#bdc3c7',
    guide = guide_colorbar(
      frame.colour = 'black', ticks.colour = 'black', title.position = 'left',
      title.theme = element_text(size = 24, hjust = 1, angle = 90),
      label.theme = element_text(size = 18), barwidth = 0.75, barheight = 10
    )
  ) +
  coord_fixed() +
  theme_bw() +
  theme(
    axis.text.x = element_text(size = 24, angle = 90, hjust = 0),
    axis.text.y = element_text(size = 24),
    legend.position = 'right',
    panel.grid.major = element_blank()
  )

ggsave('/home/adufour/work/notebook/plots/cluster_similarity.png', height = 16, width = 17)

Warning message:
“Removed 136 rows containing missing values (geom_text).”
